# Data Validation

For more details and information about data validation, see the documentation of [lamindb](https://lamin.ai/docs/db/guide/data-validation).

In [ ]:
import nbproject as nb
from lnschema_core import DObject, Pipeline, Run, Storage, Usage
from lnschema_core.dev.sqlmodel import SQLModelPrefix
from lndb import settings
from sqlmodel import Session, select, Relationship, Field, ForeignKeyConstraint
from typing import Optional
from pydantic import ValidationError

import pytest

In [ ]:
# non-optional created_at field is correctly ignored (server-side autopopulation)
pipeline = Pipeline()

In [ ]:
# invalid type for created_at field is correctly raised
with pytest.raises(ValidationError):
    pipeline = Pipeline(created_at="invalid type")

In [ ]:
# relationship validation works correctly (no error raised for valid input)
run = Run(name="Test Run", pipeline=pipeline)

In [ ]:
# complex type validation works correctly (ORM functionality is maintained)
assert run.pipeline

In [ ]:
# relationship validation works correctly (strict type checking, non-model)
with pytest.raises(TypeError):
    run = Run(name="Test Run", pipeline="invalid pipeline")

In [ ]:
# relationship validation works correctly (strict type checking, valid model)
with pytest.raises(TypeError):
    dobject = DObject(source=Pipeline())

In [ ]:
# relationship validation works correctly (strict type checking, resolved ForwardRef)
class DObjectRel(SQLModelPrefix, table=True):
    id: str = Field(primary_key=True, default=None)
    dobject_id: Optional[str] = Field(foreign_key="core.dobject.id", index=True)

    dobject: Optional["DObject"] = Relationship()


dobject = DObject(name="mydobject", run_id="raf", storage_id="g4s")
dobject_real = DObjectRel(dobject=dobject)

In [ ]:
# auto-population of required fk constraints works correctly
class DObjectRelFK(SQLModelPrefix, table=True):
    id: str = Field(primary_key=True, default=None)
    dobject_id: str = Field(foreign_key="core.dobject.id", index=True)

    dobject: "DObject" = Relationship()


dobject = DObject(name="mydobject", run_id="raf", storage_id="g4s")
dobject_rel = DObjectRelFK(dobject=dobject)

# auto-population of required fk constraints works correctly (composite fk)
class PipelineRelFK(SQLModelPrefix, table=True):
    __table_args__ = (ForeignKeyConstraint(["pipeline_id", "pipeline_v"], ["core.pipeline.id", "core.pipeline.v"]),)
    id: str = Field(primary_key=True, default=None)
    pipeline_id: str = Field(index=True)
    pipeline_v: str = Field(index=True)

    pipeline: "Pipeline" = Relationship()


pipeline = Pipeline(name="mypipeline")
pipeline_rel = PipelineRelFK(pipeline=pipeline)

In [ ]:
# dobject validation works correctly (missing field errors are raised even when dobject is instantiated with custom constructor)
with pytest.raises(ValidationError):
    dobject = DObject(name="no data objects")

In [ ]:
# dobject validation works correctly (no errors raised when dobject is conformantly instantiated with custom constructor)
dobject = DObject(name="mydobject", run_id=nb.meta.store.id, storage_id="123")

In [ ]:
# enumeration validation works correctly (no errors raised when a valid enumeration member is passed)
assert Usage(type="ingest", dobject_id=dobject.id)

In [ ]:
# categorical validation works correctly (error raised when invalid enumeration member is passed)
with pytest.raises(ValidationError):
    Usage(type="invalid member", dobject_id=dobject.id)